In [1]:
import time #import time module to make some delay
import os
import cv2 #import cv2 module to capture frame from camera
from keras.models import load_model # import load model to load a pretrained model
from cvzone.HandTrackingModule import HandDetector # hand detector module to detect hand
import numpy as np # import numpy to make numerical calculations
import math # import module to make mathematecial calculations
import datetime # to get system date and time

In [2]:
class_list=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',' '] # defining all classes

In [3]:
from keras.models import Sequential,load_model
from keras.layers import Dense
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=15))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(len(class_list), activation='softmax'))
# Compile the model
model.compile(optimizer='adam', 
            loss='categorical_crossentropy', 
            metrics=['accuracy'])

In [4]:
model.load_weights(r"C:\Users\Admin\Downloads\HAND GESTURE WRITING-2\model\SavedModel.h5")
cap=cv2.VideoCapture(0) #making a capture instance to activate webcam
cap.set(3,1280) # setting video window width 1280
cap.set(4,740) # setting video window height 780
detector=HandDetector(detectionCon=0.4,maxHands=1) #initialize hand detector module
finalstring=""
match_count=0
match_thresold=15

In [5]:
def calculate_distance(landmark_list,a,b): #FUNCTION TO CLACULATE DISTANCE BETWEEN TWO POINT OF FINGER
    landmark1,landmark2=landmark_list[a],landmark_list[b]
    p1=(landmark1[0],landmark1[1])
    p2=(landmark2[0],landmark2[1])
    dist=int(math.dist(p1,p2))                  
    return dist

In [ ]:
while True:
    sttime=time.time() #start time to calculate fps
    success,frame=cap.read() #reading live frame
    if success: #if we get frame from camera
        frame=cv2.flip(frame,1) #fliping horizontaly to avoid mirror image
        frame_orig=frame.copy() #take a copy of original for saving
        cv2.putText(frame,f"Make gesture inside green box",(800,80),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(255,0,255),1,cv2.LINE_AA)
        cv2.rectangle(frame,(800,100),(1200,600),(0,255,0),2,cv2.LINE_AA) #drawing a rectangle on screen
        hands,frame=detector.findHands(frame,flipType=False,draw=False) #detect hand
        if hands:
            lmlist=hands[0]['lmList'] #get hand points
            X_train=[] #create the input features
            x1=calculate_distance(lmlist,4,8) # get stance between point 4 and 8
            x2=calculate_distance(lmlist,8,12) # get stance between point 8 and 12
            x3=calculate_distance(lmlist,12,16)
            x4=calculate_distance(lmlist,16,20)
            x5=calculate_distance(lmlist,0,4)
            x6=calculate_distance(lmlist,0,8)
            x7=calculate_distance(lmlist,0,12)
            x8=calculate_distance(lmlist,0,16)
            x9=calculate_distance(lmlist,0,20)
            x10=calculate_distance(lmlist,4,5)
            x11=calculate_distance(lmlist,8,5)
            x12=calculate_distance(lmlist,12,9)
            x13=calculate_distance(lmlist,16,13)
            x14=calculate_distance(lmlist,20,17)
            x15=calculate_distance(lmlist,5,17)
            X_train.append([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15]) #append all input feaures in list
            x_test=np.array(X_train) #making the input features to feed into model
            pred=model.predict(x_test) #prediction with the model
            pred_indx=np.argmax(pred[0]) #get the index of maximum confidence
            final=class_list[pred_indx] # get the alphabet of maximum confidence
            confidence=np.round(pred[0][pred_indx]*100,2) #get the confidence percentange
            if confidence>85: #if confidence greater than 95% then only we consider
                match_count+=1
                cv2.rectangle(frame,(0,380),(500,410),(255,255,255),-1,cv2.LINE_AA) #drawing a rectangle as background of text
                cv2.putText(frame,f"Matching {final} {match_count} out of {match_thresold}",(0,400),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,0),1,cv2.LINE_AA) #show the final string
                if match_count>match_thresold:
                    match_count=0
                    finalstring=finalstring+final #making string to display
                    if len(finalstring)>30: #keeping string length fix so than dont go out of screen
                        finalstring=""
            else:
                match_count=0
        else:
            match_count=0
        endtime=time.time() #end time to calculate fps
        fps=int(1/(endtime-sttime)) #fps calculation
        cv2.putText(frame,f"FPS:{fps}",(0,30),cv2.FONT_HERSHEY_PLAIN,2,(255,0,255),2,cv2.LINE_AA) #show fps
        cv2.putText(frame,finalstring,(0,200),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(255,0,0),1,cv2.LINE_AA) #show the final string
        cv2.imshow("image",frame) #show frame in window
        k=cv2.waitKey(1) #wait for 1ms 
        if k==27:    # Esc key to stop
            break
    else:
        print("No camera found")
cv2.destroyAllWindows() #to destroy all cv2 window

1/1 [==============================] - 0s 16ms/step
